In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime
import glob

import pandas as pd

In [ ]:
## delete the folders not in live config ( extra crap )
live_config_files = glob.glob('cfgs_live/*', recursive=True)
allowed_symbols = []
symbol_files = {}
for f in live_config_files:
    symbol = f.split('\\')[-1]
    symbol = symbol.replace('.json', 'USDT')
    allowed_symbols.append(symbol)
    symbol_files[symbol] = f

# static_configs = {}
# for symbol in symbol_files:
#     file = symbol_files[symbol]
#     with open(file) as f:
#         data = json.load(f)
#     config_name = data['config_name']
#
#     if 'static' in config_name:
#         t = os.path.getmtime(file)
#         modification_time = datetime.datetime.fromtimestamp(t)
#         static_configs[symbol] = modification_time
#
# #print (static_configs)
# old_configs_if_before = datetime.datetime(2022, 3, 31)
# static_configs_new = [x for x in static_configs if static_configs[x] < old_configs_if_before]
#
# #print(static_configs_new)
# x = ''
# for c in static_configs_new:
#     x += c+'\n'
# with open('next_batch_config','w') as f:
#     f.write(x)
#
# #allowed_symbols = ["ALICEUSDT","XRPUSDT","HOTUSDT"]

In [ ]:
# ## NOW copy result folders :)
# import json
# import shutil
#
# files = glob.iglob('backtests/binance/' + '**/live_config.json', recursive=True)
#
# for filename in files:
#     with open(filename) as f:
#         config = json.load(f)
#     symbol = config['config_name'].split('_')[2]
#
#     if symbol not in symbol_files: continue
#
#     with open(symbol_files[symbol]) as f:
#         production_config = json.load(f)
#
#     long_pc = production_config['long']
#     long_c = config['long']
#
#     k_pc = long_pc.keys()
#     k_c = long_c.keys()
#
#     if len(k_c) != len(k_c): continue
#
#     same = True
#     for k in k_pc:
#         if k not in k_c:
#             same = False
#             break
#         v_pc = long_pc[k]
#         v_c = long_c[k]
#         #print(f'{k}={v_c},{v_pc}')
#
#     if not same: continue
#
#     print(f'{filename} is same as {symbol_files[symbol]}')
#
#     src = filename.replace('\\live_config.json', '')
#     dst = f'backtests/binance_production/{symbol}'
#     shutil.copytree(src=src, dst=dst, dirs_exist_ok=True)



In [ ]:
import requests
prices_url = "https://fapi.binance.com/fapi/v1/ticker/price"
pricedata = requests.get(prices_url).json()
volume_url = "https://fapi.binance.com//fapi/v1/ticker/24hr"
volume_data = requests.get(volume_url).json()
data_url = "https://fapi.binance.com/fapi/v1/exchangeInfo"
data = requests.get(data_url).json()

symbol_notional_volume = {}

for datas in data["symbols"]:
    if "USDT" in datas["pair"]:
        ts = datas['onboardDate']
        listed_on = datetime.datetime.utcfromtimestamp(ts / 1000).strftime(
            '%Y-%m-%d %H:%M:%S')
        symbol = datas["pair"]

        if symbol not in allowed_symbols:
            continue

        min_qty = float(datas["filters"][1]["minQty"])
        min_notional_fixed = float(datas["filters"][5]["notional"])

        price = 0
        for i in pricedata:
            if i["symbol"] == symbol:
                price = i["price"]

        for i in volume_data:
            if i["symbol"] == symbol:
                volm = str((int(float(i["quoteVolume"]) / 1000000))).zfill(4)

        min_notional_calc = min_qty * float(price)

        if min_notional_calc <= min_notional_fixed:
            min_notional = min_notional_fixed
        else:
            min_notional = min_notional_calc

        symbol_notional_volume[symbol] = {
            "vol":volm,
            "listed":listed_on,
            "notion":min_notional
        }

symbol_notional_volume


In [ ]:
## get current config
current_config = pd.read_csv('run_config.csv')

config_data = {}
for symbol in allowed_symbols:
    c = current_config.loc[current_config['symbol'] == symbol.replace('USDT','')]
    config_data[symbol]={
        'lm':c['long_mode'].values[0],
        'sm':c['short_mode'].values[0],
        'le':c['long_exposure'].values[0],
        'se':c['short_exposure'].values[0],
    }
config_data

In [ ]:
import warnings

warnings.filterwarnings("ignore")

big_data = pd.DataFrame()
files = glob.iglob('backtests/binance_production/' + '**/backtest_result.txt', recursive=True)
i = 0
#print(symbol_notional_volume)
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance_production', '')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value
                if key == 'Production Config':
                    d['prod'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'long':
                    d['maxsl'] = float(value.replace('%', '').strip()) / 24
                if key == 'Price action distance max' and current_state == 'long':
                    d['pamaxl'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'long':
                    d['pastdl'] = float(value.strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'short':
                    d['maxss'] = float(value.replace('%', '').strip()) / 24
                if key == 'Price action distance max' and current_state == 'short':
                    d['pamaxs'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'short':
                    d['pastds'] = float(value.strip())

        # # symbol_notional_volume[symbol] = {
        # #     "vol":volm,
        # #     "listed":listed_on,
        # #     "notion":min_notional
        # # }
        symbol = d['s']
        d['vol'] = symbol_notional_volume[symbol]['vol']
        d['listed'] = symbol_notional_volume[symbol]['listed']
        d['notion'] = symbol_notional_volume[symbol]['notion']
        # # config_data[symbol]={
        # #     'lm':c['long_mode'].values[0],
        # #     'sm':c['short_mode'].values[0],
        # #     'le':c['long_exposure'].values[0],
        # #     'se':c['short_exposure'].values[0],
        # # }
        d['lm'] = config_data[symbol]['lm']
        d['sm'] = config_data[symbol]['sm']
        d['le'] = config_data[symbol]['le']
        d['se'] = config_data[symbol]['se']
        # print(d)
        big_data = big_data.append(d, ignore_index=True)

big_data

Now filter the dataframe

In [ ]:
#big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%'] if
df = big_data[(big_data.s.isin(allowed_symbols)) ]
df

In [ ]:
# df.drop_duplicates(
#     subset=['s', 'prod', 'tgl_%', 'tgs_%', 'adgl_%', 'adgs_%', 'adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long',
#             'short'], keep='first',
#     inplace=True)
dx = df[['s','lm','sm','notion','le','se','vol', 'adgl_%', 'adgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'maxsl', 'pamaxl','pastdl',
         'maxss', 'pamaxs','pastds' ]]
dx

In [ ]:
ds = dx[(dx['sm'] == 'n') | (dx['sm'] == 'gs')]
ds

In [ ]:
ds_proposed = dx[
    #(dx['sm'] == 'n') &
    (dx['bnkrs_%'] == 100) &
    # (dx['ebrs'] > 0.7) &
    (dx['pastds'] < 0.1)
    #(dx['maxss'] < 35)
    ]
ds_proposed = ds_proposed[['s','sm','notion','le','se','vol', 'adgs_%', 'bnkrs_%',  'ebrs','maxss', 'pamaxs','pastds' ]]
ds_proposed

ds = dx[
   (dx['adgs_%'] > 0.00)
  #& (dx['bnkrs_%'] == 100.00)
]
ds = ds[['file', 'prod', 's', 'adgs_%', 'bnkrs_%','ebrs','maxss', 'pamaxs','pastds' ]]
ds

In [ ]:
short_symbols = ds['s'].tolist()
print(short_symbols)
config = pd.read_csv('run_config.csv')
config['usdt'] = config['symbol']+'USDT'
config = config[(config.usdt.isin(short_symbols))]
config

## find the ones that are not supposed to be shorted!
short_symbols = ds['s'].tolist()

config = pd.read_csv('run_config.csv')
config['usdt'] = config['symbol']+'USDT'
config = config[(~config.usdt.isin(short_symbols)) & (config['short_mode'] == 'n')]
## find the risky ones!
not_supposed_to_be_shorted = config['usdt'].tolist()
not_supposed_to_be_shorted

dx
# ss = dx['s'].tolist()
# ss
ds_not_good = dx[(dx.s.isin(not_supposed_to_be_shorted))]
#s_not_good
#ds_not_good
ds_not_good